<a href="https://colab.research.google.com/github/kozyreviva/Data_Science/blob/main/ML_introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>Дописал) Спасибо за подсказку.
</div>

**ВВЕДЕНИЕ В МАШИННОЕ ОБУЧЕНИЕ**

Оператор мобильной связи «Мегалайн» хочет построить систему, способную проанализировать поведение клиентов и предложить пользователям новые тарифы: «Смарт» или «Ультра».

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф.

Необходимо постройте модель с максимально большим значением accuracy. Нужно довести долю правильных ответов по крайней мере до 0.75.



**План работы**
 * Открыть файл с данными и изучить его. Путь к файлу: datasets/users_behavior.csv. Скачать датасет.
 * Разделить исходные данные на обучающую, валидационную и тестовую выборки.
 * Исследовать качество разных моделей, меняя гиперпараметры. Кратко написать выводы исследования.
 * Проверить качество модели на тестовой выборке.
 * Дополнительное задание: проверить модель на вменяемость.

Подключение библиотек.

In [71]:
#подключение библиотек
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

In [72]:
#датафрейм сохраняем в переменную
try:
  df = pd.read_csv("/datasets/users_behavior.csv")
except:
  df = pd.read_csv("users_behavior.csv")

Просмотр датафрейма.

In [73]:
#вывод первых 5 строк датафрейма
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Для построения обучающих моделей необходимо разделить датафрейм на признаки и целевой признак. 

Запишим их в переменные.

In [74]:
features = df.drop("is_ultra", axis = 1)
target = df["is_ultra"]

Для обучения моделей разделим поделим признаки на 3 части. 

* Тренировочную - для обучения модели
* Валидационную - для проверки подели
* Тестовую - для оценки готовой модели

Т.к. мы пользуемся одним набором данных, то поделим признаки сначало в соотношении 80/20/20

In [75]:
features_train, features_test, target_train, target_test = train_test_split(features, target,
                                                                                 random_state = 12345, test_size = 0.4)

Поделим тестовую выборку пополам.

In [76]:
features_test, features_valid, target_test, target_valid = train_test_split(features_test, target_test,
                                                                             random_state = 12345,test_size = 0.5)

Подберем наилучшую модель для более точного прогноза.
Чтобы оценить качество модели для задач классификации будет пользоваться  метрикой accuracy_score.


Деерево решений.
Зададим гиперпараметры. Для псевдослучайности random_state зададим 12345. Глубину дерева max_depth подберем в цикле для того чтобы избежать недообучения и переобучения. Выведем наилучшую точность и гиперпараметр.

In [77]:

best_tree_depht = 0
best_accuracy = 0


for depth in range(1,11):
        model = DecisionTreeRegressor(random_state = 12345,max_depth = depth)
        model.fit(features_train, target_train)
        predictions = model.predict(features_valid)
        accuracy = accuracy_score(target_valid, predictions.round())
        if accuracy > best_accuracy:
            best_accuracy = accuracy 
            best_tree_depth = depth
            best_model = model


print(best_accuracy)
print(best_tree_depth)



0.7931570762052877
7


Случайный лес.
Зададим гиперпараметры. Для псевдослучайности random_state зададим 12345. Глубину дерева max_depth подберем в цикле от 1 до 10. Количество деревьев подберем в цикле с шагом 10. Выведем наилучшую точность и гиперпараметры.

In [78]:

best_forest_depht = 0
best_nest = 0
best_accuracy = 0


for est in range(10, 51, 5):
    for depth in range(1,11):
        model = RandomForestRegressor(random_state = 12345,n_estimators = est, max_depth = depth)
        model.fit(features_train, target_train)
        predictions = model.predict(features_valid)
        accuracy = accuracy_score(target_valid,predictions.round())
        if accuracy>best_accuracy:
            best_accuracy =accuracy
            best_forest_depth = depth
            best_nest = est


print(best_accuracy)
print(best_forest_depth)
print(best_nest)


0.8118195956454122
9
15


Линейная регрессия.

Выведем точность модели.

In [79]:



model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
accuracy = accuracy_score(target_valid,predictions.round())
        
print(accuracy)



0.7402799377916018


**Выводы**

Как видим,  модель "Случайный лес" показала наибольшую точность в 81%. Количество деревьев 15 и глубина 9, такие гиперараметры дали наилучший результат. 

Оценим готовую модель случайного леса на тестовой выборке.

In [80]:
model = RandomForestRegressor(random_state = 12345, n_estimators = 15, max_depth = 9)
model.fit(features_train,target_train)
predictions = model.predict(features_test)

print(accuracy_score(target_test,predictions.round()))

0.7869362363919129


Предсказанные значения точны на 78% с реальными значениями.